In [33]:
from scipy.io.wavfile import read, write
import torchaudio
import torch
from librosa.util import normalize
from librosa.filters import mel as librosa_mel_fn
import numpy as np
import librosa
import librosa.display
from tqdm import tqdm
import os
import soundfile as sf
import matplotlib.pyplot as plt

In [34]:
MAX_WAV_VALUE = 32768.0

def load_wav(full_path):
    sampling_rate, data = read(full_path)
    return data, sampling_rate

def dynamic_range_compression(x, C=1, clip_val=1e-5):
    return np.log(np.clip(x, a_min=clip_val, a_max=None) * C)

def dynamic_range_decompression(x, C=1):
    return np.exp(x) / C

def dynamic_range_compression_torch(x, C=1, clip_val=1e-5):
    return torch.log(torch.clamp(x, min=clip_val) * C)

def dynamic_range_decompression_torch(x, C=1):
    return torch.exp(x) / C

def spectral_normalize_torch(magnitudes):
    output = dynamic_range_compression_torch(magnitudes)
    return output

def spectral_de_normalize_torch(magnitudes):
    output = dynamic_range_decompression_torch(magnitudes)
    return output

mel_basis = {}
hann_window = {}

def mel_spectrogram(y, n_fft, num_mels, sampling_rate, hop_size, win_size, fmin, fmax, center=False):
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))

    global mel_basis, hann_window
    if fmax not in mel_basis:
        mel = librosa_mel_fn(sr=sampling_rate, n_fft=n_fft, n_mels=num_mels, fmin=fmin, fmax=fmax)
        mel_basis[str(fmax)+'_'+str(y.device)] = torch.from_numpy(mel).float().to(y.device)
        hann_window[str(y.device)] = torch.hann_window(win_size).to(y.device)

    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=hann_window[str(y.device)],
                      center=center, pad_mode='reflect', normalized=False, onesided=True)

    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9))

    spec = torch.matmul(mel_basis[str(fmax)+'_'+str(y.device)], spec)
    spec = spectral_normalize_torch(spec)

    return spec

In [35]:
wav_path = "/blob/v-yuancwang/DiffAudioImg/VGGSound/data/vggsound/wav"
sr_wav_path = "/blob/v-yuancwang/DiffAudioImg/VGGSound/data//vggsound/sr/wav"
sr_mel_path = "/blob/v-yuancwang/DiffAudioImg/VGGSound/data//vggsound/sr/mel"

In [36]:
wav_files = os.listdir(wav_path)
len(wav_files)

177056

In [43]:
sr_mel_files = os.listdir("/blob/v-yuancwang/DiffAudioImg/VGGSound/data//vggsound/sr/mel")
len(sr_mel_files)

177048

In [41]:
sr_wav_files = os.listdir(sr_wav_path)
sr_wav_set = set(sr_wav_files)
len(sr_wav_set)

177056

In [ ]:
for file_name in sr_mel_files[:]:
    mel = np.load(os.path.join(sr_mel_path, file_name))
    if mel.shape[1] < 624:
        # print(mel.shape)
        mel = np.pad(mel, ((0, 0), (0, 624 - mel.shape[1])), 'wrap')
        np.save(os.path.join(sr_mel_path, file_name), mel)